In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import psutil

import pandas as pd
import numpy as np

import time
from datetime import datetime
import threading


In [ ]:
def kill_background_task(task_name):
    # 遍历所有进程
    for proc in psutil.process_iter(['pid', 'name']):
        # 检查进程名称是否匹配
        if task_name in proc.info['name']:
            # 终止进程
            proc.terminate()

def config(urls):
    user_data_dir = r"C:\Users\haokw\AppData\Local\Google\Chrome\User Data"
    service = webdriver.chrome.service.Service(ChromeDriverManager().install())
    options = webdriver.ChromeOptions()
    options.add_argument(f"user-data-dir={user_data_dir}")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920x1080")
    options.add_argument("--ignore-certificate-errors")  # 忽略 SSL 错误
    options.add_argument("--log-level=3")  # 设定日志等级
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Your User-Agent")

    kill_background_task("chrome")
    driver = webdriver.Chrome(options=options, service=service)
    driver.execute_cdp_cmd('Page.addScriptToEvaluateOnNewDocument', {
        'source': '''
            Object.defineProperty(navigator, 'webdriver', {
                get: () => undefined
            })
        '''
    })
    driver.get(urls)
    time.sleep(1)
    return driver

# 登录函数
def login(driver, username, password):
    try:
        # 等待用户名输入框加载并输入用户名
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "#formLogin > div:nth-child(1) > div > div > input")
            )
        ).send_keys(username)
        
        # 等待密码输入框加载并输入密码
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "#formLogin > div.el-form-item.form-item-class > div > div > input")
            )
        ).send_keys(password)
        
        print("用户名和密码已输入")
        
        # 点击登录按钮
        login_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "#formLogin > button.el-button.el-button--primary.el-button--large.login-button")
            )
        )
        login_button.click()
        print("登录按钮已点击")
    except Exception as e:
        print(f"登录时发生错误: {e}")

def extract_data(driver):
    try:
        button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, "#app > div > div.main-container > section > div.homepage > div > div.card.board.r2 > div.title > div")
            )
        )
        button.click()
        # print("按钮已点击")
        
        selectors = [
            "body > div:nth-child(8) > div > section > div > div > div.mainbox > div > div:nth-child(1) > div:nth-child(2) > div",
            "body > div:nth-child(8) > div > section > div > div > div.mainbox > div > div:nth-child(2) > div:nth-child(2) > div",
            "body > div:nth-child(8) > div > section > div > div > div.mainbox > div > div:nth-child(3) > div:nth-child(2) > div",
            "body > div:nth-child(8) > div > section > div > div > div.mainbox > div > div:nth-child(4) > div:nth-child(2) > div"
        ]
        
        data = []
        for selector in selectors:
            element = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, selector))
            )
            data.append(element.text)
        
        current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        return {"time": current_time, "data": data}
    except Exception as e:
        print(f"提取数据时发生错误: {e}")
        return None
    





In [ ]:
def scheduled_task(driver, results, count, max_count, last_time):
    """ 定时调用数据提取函数 """
    if count < max_count:
        # 获取数据并保存
        data_entry = extract_data(driver)
        if data_entry:
            results.append(data_entry)
            print(f"次数：{count},数据记录: {data_entry}")
        
        # 刷新当前页面
        driver.refresh()

        # 计算下一次执行的准确时间
        current_time = time.time()
        next_time = last_time + 180  # 设置下一次任务的执行时间（10秒间隔）
        
        # 计算剩余时间并设置定时器
        delay = next_time - current_time
        if delay > 0:
            # 如果下一次任务还没到，设置定时器
            threading.Timer(delay, scheduled_task, [driver, results, count + 1, max_count, next_time]).start()
        else:
            # 如果当前时间已超过下一次任务的时间，立即执行
            scheduled_task(driver, results, count + 1, max_count, next_time)
    else:
        # 达到最大调用次数，停止定时任务
        print("任务完成，停止定时器。")


In [ ]:
def main():
    urls = "http://139.198.191.254:8070/#/login?redirect=/"
    driver = config(urls)
    username = "zjdx"  # 替换为实际用户名
    password = "OeU14zS3!q"  # 替换为实际密码

    time.sleep(5)
    print('浏览器配置完成')

    # 登录
    login(driver, username, password)
    
    # 数据保存
    results = []  # 用于存储结果的列表
    max_count = 50  # 最大调用次数

    # 获取当前时间作为开始时间
    start_time = time.time() + 10

    # 启动定时任务
    scheduled_task(driver, results, 0, max_count, start_time)

    # 让主线程持续运行，直到任务完成
    while len(results) < max_count:
        time.sleep(1)  # 保持主线程运行

    # 任务完成后，保存结果到 Excel
    print("所有数据记录:")
    # print(results)
    return results



In [ ]:
results = main()


In [ ]:
data_array = np.array([entry['data'] for entry in results])  # 转换为 NumPy 数组形式
# 将所有 'data' 转换为 NumPy 数组
# 提取所有时间
time_entries = [entry['time'] for entry in results]  # 提取时间列表

print("所有数据的 NumPy 数组：")
# print(data_array)
print("所有时间：")
# print(time_entries)


In [ ]:
import pandas as pd

# 将数据转换为 DataFrame
df = pd.DataFrame(data_array, columns=["催化净化", "废水净化", "吸附净化", "废水监测"])
df["获取时间"] = time_entries  # 添加时间列
print(df)
# 保存到 Excel
df.to_excel("results.xlsx", index=False)

print("数据已保存到 results.xlsx")
